In [ ]:
import numpy as np


In [ ]:
from docplex.mp.model import Model
import time
import numpy as np
def create_bpp(weights,bin_size):
    model = Model('BinPacking')
    n = len(weights)  # number of items
    m = n  # number of bins(worst case each item occupies one  bin)
    #Decision variable
    # x[i,j] = 1 if i occupies bin j
    x= model.binary_var_matrix(n,m,name='x', key_format='item_{0}_bin_{1}')
    # y[j]= 1 is bin j is used
    y=model.binary_var_list(m,name='y', key_format='bin_{0}') 
    # Objective: Minimize number of bins used
    model.minimize(model.sum(y[j] for j in range(m)))
    #Constraints:
    for i in range(n):
         model.add_constraint(
            model.sum(x[i,j] for j in range(m)) == 1,
            f'item_{i}_assigned'
        )
       

    for j in range(m):
         model.add_constraint(
            model.sum(weights[i] * x[i,j] for i in range(n)) <= bin_size * y[j],
            f'bin_{j}_capacity'
        )
    
    return model, x, y   

    

In [ ]:
import numpy as np

def ilp_to_qubo(model, x, y, weights, bin_size, A, B, C):
  
    n = len(weights)  # number of items
    m = n  # maximum number of bins
    Q = np.zeros((n*m + m, n*m + m))
    
    # Create variable to index mapping
    var_to_index = {}
    for i in range(n):
        for j in range(m):
            var_to_index[f'item_{i}_bin_{j}'] = i * m + j
    for j in range(m):
        var_to_index[f'bin_{j}'] = n * m + j
    
    # Process constraints from the model
    for constraint in model.iter_constraints():
        # Get left and right expressions
        left_expr = constraint.get_left_expr()
        right_expr = constraint.get_right_expr()
        sense=constraint.sense
        # Get the terms from the left expression
        terms = []
        for term in left_expr.iter_terms():
            var, coef = term[0], term[1]  # Unpack term tuple correctly
            var_name = var.get_name()
            if var_name in var_to_index:
                idx = var_to_index[var_name]
                terms.append((idx, coef))
        
       # Add quadratic penalties based on constraint type
        if sense == '==':  # equality constraint (one bin per item)
            # (sum x_ij - 1)^2 penalty
            for idx1, coef1 in terms:
                Q[idx1, idx1] += B
                for idx2, coef2 in terms:
                    if idx1 < idx2:
                        Q[idx1, idx2] += 2 * B
                        Q[idx2, idx1] += 2 * B
            
            # Add constant term to complete (sum x_ij - 1)^2
            for idx1, _ in terms:
                Q[idx1, idx1] -= 2 * B
        
        elif sense == '<=':  # inequality constraint (bin capacity)
            rhs = float(right_expr.constant if hasattr(right_expr, 'constant') else right_expr)
            
            # Add capacity constraint penalties
            for idx1, coef1 in terms:
                for idx2, coef2 in terms:
                    Q[idx1, idx2] += C * coef1 * coef2
                    
            # Add interaction with bin usage variable
            bin_idx = next((idx for idx, coef in terms if 'bin_' in var_to_index.keys()[idx]), None)
            if bin_idx is not None:
                for idx, coef in terms:
                    if idx != bin_idx:
                        Q[idx, bin_idx] -= C * coef * rhs
                        Q[bin_idx, idx] -= C * coef * rhs
                
                # Add squared term for bin capacity
                Q[bin_idx, bin_idx] += C * rhs * rhs
    
    # Add objective function (minimize number of bins)
    for j in range(m):
        Q[n*m + j, n*m + j] += A
        
    # Add additional penalties to ensure items are assigned
    for i in range(n):
        penalty = 1000  # Large penalty for not assigning items
        row_sum = 0
        for j in range(m):
            idx = var_to_index[f'item_{i}_bin_{j}']
            row_sum += 1
            Q[idx, idx] += penalty
        
        # Subtract penalty for correct assignment
        for j in range(m):
            idx = var_to_index[f'item_{i}_bin_{j}']
            Q[idx, idx] -= 2 * penalty * row_sum
            for k in range(j+1, m):
                idx2 = var_to_index[f'item_{i}_bin_{k}']
                Q[idx, idx2] += 2 * penalty
                Q[idx2, idx] += 2 * penalty
    
    return Q

In [ ]:
def cost_fun(Q,soln):
     return np.dot(soln,np.dot(Q,soln))  

In [16]:
#brute force solution
import numpy as np
def brute_force(Q,n,m):
     total_var= n*m+m #Total number of variables: n*m decision variables + m bin usage variables
     min_cost=float('inf')
     best_soln= None
     for i in range(2**total_var):
         binary_com= list(format(i, f'0{total_var}b'))
         soln = np.array([int(x) for x in (binary_com)])
         cost= cost_fun(Q,soln)
         if cost< min_cost:
             min_cost=cost
             best_soln =soln
     return best_soln,min_cost
         

In [ ]:
import time
def instance():
    np.random.seed(45)
    bin_size = 15
    A, B, C = 1, 100, 100  # Penalty coefficients

    
    instances = {
        "small": np.random.randint(1, 10, size=3),
        "medium": np.random.randint(1, 20, size=5),
        "large": np.random.randint(1, 50, size=7)
    }

    for instance_name, weights in instances.items():
        print(f"\n--- {instance_name.capitalize()} Instance ---")
        print(f"Weights: {weights}")
        
        # create ILP model
        model, x, y = create_bpp(weights, bin_size)
        
        # Transform ILP to QUBO using actual model constraints
        Q = ilp_to_qubo(model, x, y, weights, bin_size, A, B, C)
        n = len(weights)
        m = n
        
        # Solve using brute force
        start_time = time.time()
        best_soln, min_cost = brute_force(Q,n,m)
        time_taken = time.time() - start_time
        
        print(f"Best solution: {best_soln}")
        print(f"Minimum cost: {min_cost}")
        print(f"Time taken for brute force: {time_taken:.6f} seconds")

if __name__ == "__main__":
    instance()


--- Small Instance ---
Weights: [4 1 6]
Best solution: [0 1 1 0 1 1 0 1 1 0 0 0]
Minimum cost: -18000.0
Time taken for brute force: 0.029085 seconds

--- Medium Instance ---
Weights: [ 4  5 16  2 15]
Best solution: [0 0 1 1 1 0 0 1 1 1 0 0 1 1 1 0 0 1 1 1 0 0 1 1 1 0 0 0 0 0]
Minimum cost: -75000.0
Time taken for brute force: 6521.872482 seconds

--- Large Instance ---
Weights: [ 9 24 47  9 13 35 25]
